In [1]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\hp\\Documents\\Projects\\Python\\SkinCancer\\SkinCancer-Classification'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
        

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path= CONFIG_FILE_PATH,
        params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
        
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(path_of_model="artifacts/training/model.keras",
                                      training_data="artifacts/data_ingestion",
                                      all_params=self.params,
                                      params_image_size=self.params.IMAGE_SIZE,
                                      params_batch_size=self.params.BATCH_SIZE)
        return eval_config

In [12]:
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(rescale = 1./255,
                                   validation_split=0.30)
        
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1],
                              batch_size=self.config.params_batch_size,
                              interpolation="bilinear")
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(directory=self.config.training_data,
                                                                      subset="validation",
                                                                      shuffle=False,
                                                                       **dataflow_kwargs)
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
        
        

In [14]:
try: 
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e

[2024-10-16 18:05:49,729: INFO: common: Reading the yaml file from config\config.yaml]
[2024-10-16 18:05:49,756: INFO: common: Reading the yaml file from params.yaml]
[2024-10-16 18:05:49,778: INFO: common: Directory created at artifacts]
Found 964 images belonging to 2 classes.


C:\Users\hp\anaconda3\envs\skin\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 600s 10s/step - accuracy: 0.8428 - loss: 1.2357
[2024-10-16 18:15:54,650: INFO: common: Json file saved at scores.json]
